# Master Thesis

### Description

# Github

In [10]:
import requests
from github import Github
import getpass
import time
from IPython.display import clear_output, Markdown, display

In [11]:
def get_file_url(comment):
    sha = repo.get_commit(comment.commit_id).sha
    file_url = repo.get_commit(sha).files.pop().raw_url
    return file_url
    

In [12]:
def display_file_content(file_url, comment_position):
    stream = requests.get(file_url, stream=True)
    i = 1
    for line in stream.iter_lines():
        if i == comment_position:
            display(Markdown(str(i)+" "+str(line)[2:-1]))
        elif (i >= comment_position - 20) and (i <= comment_position + 20): 
            print (str(i)+" "+str(line)[2:-1])
        i=i+1

In [13]:
def display_comment(comment, comment_position):
    print("-------------------------------------------------------")
    print("Comment: " + comment.body)
    print("Assigned to line: " + str(comment_position))


In [14]:
def display_assigment(comment):
    comment_position = comment.original_position
    file_url = get_file_url(comment)
    
    print(file_url)
    display_file_content(file_url, comment_position)
    display_comment(comment, comment_position)

In [15]:
def review_assignment(review_comments):
    for comment in review_comments:
        display_assigment(comment)
        time.sleep(1)
        assignment = input()
        if assignment == "quit": 
            break
        else:
            clear_output(wait=True)

In [52]:
%%javascript
 IPython.OutputArea.prototype._should_scroll = function(lines) {
     return false;
 }

<IPython.core.display.Javascript object>

## Log into your GitHub account
by inputing your credentials

In [16]:
log = input()
pwd = getpass.getpass()

Mirrvin
········


or providing a file with credentails in format "login,password". 

Name the file "credentials.txt".

In [7]:
credentials_file = open('credentials.txt', 'r')
log, pwd = credentials_file.read().split(',')

After providing credentials run the cells bellow to start reviewing Comments and their assigments.

In [17]:
g = Github(log,pwd)
repo = g.get_repo("ReactiveX/RxJava")
review_comments = repo.get_pulls_review_comments()

Na ten moment input niczego nie zmienia, ale chciałbym, żeby docelowo zapisywał czy komentarz jest dobrze przypięty, czy nie

aby przerwać należy wpisać "quit"

In [ ]:
review_assignment(review_comments)

https://github.com/ReactiveX/RxJava/raw/f0869179e97f9ba8d683ef6b0132257259e54033/language-adaptors/rxjava-clojure/build.gradle
1 buildscript {
2     repositories { maven { url "http://clojars.org/repo" } }
3     dependencies { classpath "clojuresque:clojuresque:1.5.8" }
4 }
5 
6 apply plugin: 'clojure'
7 
8 repositories {
9     clojarsRepo()
10 }
11 
12 dependencies {
13     compile 'org.clojure:clojure:1.4.+'
14     examplesCompile 'clj-http:clj-http:0.6.4' // https://clojars.org/clj-http
15 }
16 
17 aotCompile = true
18 warnOnReflection = false
19 
20 eclipse {


21   project {

22     natures "ccw.nature"
23   }
24 }
-------------------------------------------------------
Comment: I see, it's used in the examples only, fixing…

Assigned to line: 21
